## Evaluation of Deep_iSith and LSTM models on the holdout set
Four metrics are computed for both models. columnwise AUC, Precision, Recall and F1.   
Use  PyTorch 1.4.0 Py3.7 Kernal/ Container on Rivanna   
Try to use V100 GPU, since it is much faster than the others   


### pip install the necessary packages and download the [SITH_Layer_master] folder

In [37]:
#!pip install --user mne
#!pip install --user seaborn
#### pytorch and Cuda should be set up correctly on the Pytorch kernal or pytorch container

### Deep_SITH parameters and architecture

In [38]:
import warnings
warnings.filterwarnings('ignore')

from src.eval import evaluation
from src.train_util import *
from models.Deep_isith_EEG_model import *
from models.LSTM_EEG_model import *

# read config file
import configparser
import argparse

# preprocessing
import mne
import numpy as np
import math
import pandas as pd

# pytorch
import torch
import torch.nn
import torch.nn.functional as F
ttype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
labeltype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
from torch.utils.data import Dataset,DataLoader 

import matplotlib.pyplot as plt

# training 
from torch import nn as nn
from math import factorial
import random
import seaborn as sn
import os 
from os.path import join
import glob

# validation
from sklearn.metrics import roc_curve, auc, roc_auc_score, matthews_corrcoef,confusion_matrix,plot_roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score

<class 'torch.cuda.DoubleTensor'>


In [39]:
sith_params1 = {"in_features":32, 
                        "tau_min":1, "tau_max":50, 
                        "k":23, 'dt':1,
                        "ntau":10, 'g':0.0,  
                        "ttype":ttype, 
                        "hidden_size":20, "act_func":nn.ReLU()}

sith_params2 = {"in_features":sith_params1['hidden_size'], 
                        "tau_min":1, "tau_max":200.0,  
                        "k":12, 'dt':1,
                        "ntau":10, 'g':0.0, 
                        "ttype":ttype, 
                        "hidden_size":20, "act_func":nn.ReLU()}
sith_params3 = {"in_features":sith_params2['hidden_size'], 
                    "tau_min":1, "tau_max":800.0,  
                    "k":7, 'dt':1,
                    "ntau":10, 'g':0.0, 
                    "ttype":ttype, 
                    "hidden_size":20, "act_func":nn.ReLU()}
layer_params = [sith_params1, sith_params2,sith_params3]

file = 'Deep_isith_Subject2_numEvent0.pth'
PATH = 'saved_NNs/' + file
model1 = DeepSITH_Tracker(out=2,
                         layer_params=layer_params, dropout=0.1).double()
model1.load_state_dict(torch.load(PATH))
model1.eval()

DeepSITH_Tracker(
  (hs): DeepSITH(
    (layers): ModuleList(
      (0): _DeepSITH_core(
        (sith): iSITH(ntau=10, tau_min=1, tau_max=50, buff_max=150, dt=1, k=23, g=0.0)
        (linear): Linear(in_features=320, out_features=20, bias=True)
        (dense_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_func): ReLU()
      )
      (1): _DeepSITH_core(
        (sith): iSITH(ntau=10, tau_min=1, tau_max=200.0, buff_max=600.0, dt=1, k=12, g=0.0)
        (linear): Linear(in_features=200, out_features=20, bias=True)
        (dense_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_func): ReLU()
      )
      (2): _DeepSITH_core(
        (sith): iSITH(ntau=10, tau_min=1, tau_max=800.0, buff_max=2400.0, dt=1, k=7, g=0.0)
        (linear): Linear(in_features=200, out_features=20, bias=True)
        (dense_bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

### Total number of parameters for Deep_isith

In [40]:
tot = 0
for p in model1.parameters():
    tot += p.numel()
print("tot_weights", tot)

tot_weights 14622


### LSTM parameters and architecture

In [41]:
# number of parameters 
hidden_size = 25
model2 = LSTM_EEG(in_features = 32, hidden_dim = hidden_size, 
                          out_feuture = 2,num_layers =3, dropout=0.1).double()
tot = 0
for p in model2.parameters():
    tot += p.numel()
print("tot_weights", tot)
print(model2)

tot_weights 16352
LSTM_EEG(
  (lstm): LSTM(32, 25, num_layers=3, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=25, out_features=2, bias=True)
)


## Evaluate all subjects for DeepSITH and LSTM

In [42]:
def inference(model, val_loader):
    """
    Test for accuracy
    Iterate through each batch and make prediciton and calculate performance metrics
    Use **matthews correlation coeefficient** since the data are imbanlanced
    Again 
    Signals need to be in correct format. validation input: [nbatch x 1 x nFeutures x time] tensor.

    The target has dimension of [time] tensor, in which each entry should be one of the numbers in 
    {0,1,2, ... K} at any time point.  
    
    """
    inferenced_y = np.empty(0)
    ground_truth_y = np.empty(0)
    for _, (val_x, labels) in enumerate(val_loader):
        out_val = model(val_x)
        #print(out_val.shape)
        # pass through a softmax to tansform to probability on the third dimention (nbatch, seq, outFeature)
        res = torch.nn.functional.softmax(out_val, dim=2)
        #print(res.shape)
        # predict should also be the second dimension [1] to clauclate AUC
        y_pred = res[:,:,1]

        # flatten the predicted result 
        y_score = np.ndarray.flatten(y_pred.detach().cpu().numpy())

        # flatten the predicted result 
        y_true = np.ndarray.flatten(labels.detach().cpu().numpy())
        
        inferenced_y = np.concatenate((inferenced_y, y_score), axis=0)
        ground_truth_y = np.concatenate((ground_truth_y, y_true), axis=0)
    return inferenced_y,ground_truth_y


## Sith parameters. Have to be exact as training

In [43]:
sith_params1 = {"in_features":32, 
                        "tau_min":1, "tau_max":50, 
                        "k":23, 'dt':1,
                        "ntau":10, 'g':0.0,  
                        "ttype":ttype, 
                        "hidden_size":20, "act_func":nn.ReLU()}

sith_params2 = {"in_features":sith_params1['hidden_size'], 
                        "tau_min":1, "tau_max":200.0,  
                        "k":12, 'dt':1,
                        "ntau":10, 'g':0.0, 
                        "ttype":ttype, 
                        "hidden_size":20, "act_func":nn.ReLU()}
sith_params3 = {"in_features":sith_params2['hidden_size'], 
                    "tau_min":1, "tau_max":800.0,  
                    "k":7, 'dt':1,
                    "ntau":10, 'g':0.0, 
                    "ttype":ttype, 
                    "hidden_size":20, "act_func":nn.ReLU()}
layer_params = [sith_params1, sith_params2,sith_params3]

## Inference_all function. Specify which model to use

In [44]:
def inference_all(model = 'DeepSITH'):
    # enable use of command line
    parser = argparse.ArgumentParser(description='Input config files')
    parser.add_argument('--config', default = 'config/training_config_Deep_isith.ini', type=str,
                        help='an integer for the accumulator')
    opt, _ = parser.parse_known_args()

    # parser to read parameters
    config = configparser.ConfigParser()
    config.sections()

    # parameters from config file
    results = []
    config.read(opt.config)
    dir = config['data']['directory']
    subject_num = int(config['data']['subject #'])
    kernel_size = int(config['training']['kernel_size'])# sliding window size to use
    step = int(config['training']['step']) #  --the step between each slice. means overlap between batches is 1- step 
    modelName = config['training']['model']
    # num of epochs to train
    nepochs = int(config['training']['nepochs'])
    loss_func =  torch.nn.CrossEntropyLoss()
    batch_size = int(config['training']['batch_size']) # batch_size is a hyper parameter to tune 
    #train_split = float(config['training']['train_split'])
    lr = float(config['training']['lr'])

    #------------------------ Preprocessing ------------------------#
    val_dir = dir + 'validation/'
    for j in range(1,13): # out loop to inference all subjects
        results = []
        subject_num = j # ignore the config subject num
        # load data and do preprocessing

        # load validation data
        print(f"Starting to load Subject{subject_num} Data.")
        for file in os.listdir(val_dir):
            sub_idx = file.find('_')
            if file[:-4].endswith('_data') & (file[4:sub_idx] == str(subject_num)):
                raw = creat_mne_raw_object(val_dir+file,read_events=True)
                # filter all channels
                input_signal,target_signal = filter_standardization(raw,window_size = 1000,
                                    l_freq = 0,h_freq = 30)

                input_tensor = ttype(input_signal.reshape(1,1,input_signal.shape[0],-1))
                target_tensor = labeltype(target_signal.reshape(6,-1)) # should be six channels
                # for batch of 1 only squeeze the first dimension
                input_tensor = input_tensor.squeeze(0)
                target_tensor = target_tensor.unsqueeze(0)
                ###########for validation do not patch data ###########
                # patches data 
                #patches_train = input_tensor.unfold(dimension = 1, size = kernel_size, step = step).permute(1,0,2)
                #patches_label = target_tensor.unfold(1, kernel_size, step).permute(1,0,2)
                #print(patches_train.shape, patches_label.shape)
                val_x_t = input_tensor
                val_y_t = target_tensor
                #test_y_t = torch.cat(train_y_list, dim=0)
                print(val_x_t.shape, val_y_t.shape)

                print("Finished! {} data are loaded and preprocessed".format(len(val_x_t)))


        # -----------------------inference ----------------------------#
        df_inf = pd.DataFrame()
        for i in range(1,7): # There are six events 1 - 6
            nClass = i - 1
            if model == 'DeepSITH':
                # make a copy of every dict don't want to change them
                layer_params_l = [sith_params1.copy(), 
                                  sith_params2.copy(),sith_params3.copy()]
                
                # j is the current subject
                file = f'Deep_isith_Subject{j}_numEvent{nClass}.pth'
                PATH = 'saved_NNs/' + file
                model1 = DeepSITH_Tracker(out=2,
                                         layer_params=layer_params_l, dropout=0.1).double()
                
            elif model == 'LSTM':
                file = f'LSTM_Subject{j}_numEvent{nClass}.pth'
                PATH = 'saved_NNs/' + file
                # number of parameters 
                hidden_size = 25
                model1 = LSTM_EEG(in_features = 32, hidden_dim = hidden_size, 
                                  out_feuture = 2,num_layers =3, dropout=0.1).double()
            else:
                print('No model found. Needs to be either DeepSITH or LSTM')
            
            
            model1.load_state_dict(torch.load(PATH))    
            model1.eval()
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model1.to(device)

            val_y_t_nClass = val_y_t[:,nClass,:]
            # create dataloader class
            val_dataset = EEGDataset(val_x_t ,val_y_t)

            val_loader = DataLoader(dataset= val_dataset, batch_size=1, 
                                 shuffle=False)
            y_pred, y_gt = inference(model1, val_loader)
            df_inf[str(nClass)] = y_pred


        # save as dataframes
        # rename
        colName = ['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase',
                  'LiftOff', 'Replace', 'BothReleased']
        df_inf.columns = colName
        if not os.path.exists('inference'):
            os.makedirs('inference')
 
        df_inf.to_csv('inference/' + f'{model}_Subject{j}' + '_inference' + '.csv')


In [45]:
inference_all(model = 'DeepSITH')

Starting to load Subject1 Data.
torch.Size([1, 32, 117333]) torch.Size([1, 6, 117333])
Finished! 1 data are loaded and preprocessed
Starting to load Subject2 Data.
torch.Size([1, 32, 149945]) torch.Size([1, 6, 149945])
Finished! 1 data are loaded and preprocessed
Starting to load Subject3 Data.
torch.Size([1, 32, 114946]) torch.Size([1, 6, 114946])
Finished! 1 data are loaded and preprocessed
Starting to load Subject4 Data.
torch.Size([1, 32, 121920]) torch.Size([1, 6, 121920])
Finished! 1 data are loaded and preprocessed
Starting to load Subject5 Data.
torch.Size([1, 32, 129504]) torch.Size([1, 6, 129504])
Finished! 1 data are loaded and preprocessed
Starting to load Subject6 Data.
torch.Size([1, 32, 135837]) torch.Size([1, 6, 135837])
Finished! 1 data are loaded and preprocessed
Starting to load Subject7 Data.
torch.Size([1, 32, 140424]) torch.Size([1, 6, 140424])
Finished! 1 data are loaded and preprocessed
Starting to load Subject8 Data.
torch.Size([1, 32, 124095]) torch.Size([1, 6

In [51]:
# make finale result table
def makeTable(model = 'DeepSITH'):
    colName = ['HandStart', 'FirstDigitTouch', 'BothStartLoadPhase',
                  'LiftOff', 'Replace', 'BothReleased']
    final_result = pd.DataFrame(columns=['Subject','Event','AUC', 'Precision', 'Recall', 'F1'])

    for j in range(1,13): # out loop evaluate all subjects
        # load data
        if model == 'DeepSITH':
            inf = f'inference/DeepSITH_Subject{j}_inference.csv'
        elif model == 'LSTM':
            inf = f'inference/LSTM_Subject{j}_inference.csv'
        else:
            print('No model found. Needs to be either DeepSITH or LSTM')
        gt = f'grasp-and-lift-eeg-detection/validation/subj{j}_series8_events.csv'
        df_inf = pd.read_csv(inf)
        df_gt = pd.read_csv(gt)
        df_inf = df_inf.drop(['Unnamed: 0'], axis = 1)
        #df_gt = df_gt.drop(['Unnamed: 0'], axis = 1)
        # make binary table
        threshold = .3
        df_inf_binary = df_inf.copy()
        df_inf_binary[df_inf_binary > threshold] = 1
        df_inf_binary[df_inf_binary <= threshold] = 0

        for i in colName:
            # evaluation metrics, i is each event
            #print(df_inf_binary[i])
            auc = roc_auc_score(y_true = df_gt[i] ,y_score = df_inf[i])
            p = precision_score(y_true = df_gt[i] ,y_pred = df_inf_binary[i])
            r = recall_score(y_true = df_gt[i] ,y_pred = df_inf_binary[i])
            f1 = f1_score(y_true = df_gt[i]  ,y_pred = df_inf_binary[i])

            # j is again subject, i is event
            final_result = final_result.append({'Subject':j,'Event':i,'AUC':auc, 
                                                'Precision':p, 'Recall':r, 'F1':f1}, ignore_index=True)
            print(f'AUC:{auc}, Precision:{p}, Recall:{r}, F1:{f1}')
            
    return final_result

In [52]:
final_result_SITH = makeTable(model = 'DeepSITH')

AUC:0.9421479090330811, Precision:0.8016894609814964, Recall:0.3907843137254902, F1:0.5254416029528078
AUC:0.9835303237330323, Precision:0.7523031461845994, Recall:0.8486274509803922, F1:0.7975674928591173
AUC:0.9888792485800286, Precision:0.8151408450704225, Recall:0.8170588235294117, F1:0.8160987074030552
AUC:0.9700617081096871, Precision:0.6173617021276596, Recall:0.7111764705882353, F1:0.6609567198177676
AUC:0.984189685568346, Precision:0.6954342984409799, Recall:0.7347058823529412, F1:0.7145308924485126
AUC:0.9687344762288116, Precision:0.6981062012625325, Recall:0.7372549019607844, F1:0.7171466717528132
AUC:0.9543428330081041, Precision:0.5344166102459942, Recall:0.46431372549019606, F1:0.4969048368481796
AUC:0.9767026023084852, Precision:0.5454980079681275, Recall:0.6711764705882353, F1:0.6018461538461538
AUC:0.9827109601270864, Precision:0.651024208566108, Recall:0.6854901960784314, F1:0.6678127984718243
AUC:0.9183673636253495, Precision:0.39921512665001785, Recall:0.4388235294

In [53]:
final_result_SITH.to_csv(f'DeepSITH_final_result_padding.csv',index=False)

In [54]:
final_result_SITH.mean()

Subject      6.500000
AUC          0.914308
Precision    0.534818
Recall       0.447440
F1           0.472611
dtype: float64

In [55]:
final_result_SITH.groupby('Subject').aggregate({'AUC':'mean','Precision':'mean','Recall':'mean',
                                 'F1':'mean'}).round(3)

,AUC,Precision,Recall,F1
Subject,,,,
1,0.973,0.730,0.707,0.705
2,0.953,0.481,0.595,0.523
3,0.913,0.557,0.414,0.461
4,0.938,0.607,0.501,0.545
5,0.847,0.400,0.153,0.217
6,0.910,0.545,0.399,0.453
7,0.918,0.516,0.351,0.415
8,0.894,0.541,0.441,0.464
9,0.923,0.599,0.541,0.566


## Evaluation for LSTM

In [33]:
inference_all(model = 'LSTM')

Starting to load Subject1 Data.
torch.Size([1, 32, 117333]) torch.Size([1, 6, 117333])
Finished! 1 data are loaded and preprocessed
Starting to load Subject2 Data.
torch.Size([1, 32, 149945]) torch.Size([1, 6, 149945])
Finished! 1 data are loaded and preprocessed
Starting to load Subject3 Data.
torch.Size([1, 32, 114946]) torch.Size([1, 6, 114946])
Finished! 1 data are loaded and preprocessed
Starting to load Subject4 Data.
torch.Size([1, 32, 121920]) torch.Size([1, 6, 121920])
Finished! 1 data are loaded and preprocessed
Starting to load Subject5 Data.
torch.Size([1, 32, 129504]) torch.Size([1, 6, 129504])
Finished! 1 data are loaded and preprocessed
Starting to load Subject6 Data.
torch.Size([1, 32, 135837]) torch.Size([1, 6, 135837])
Finished! 1 data are loaded and preprocessed
Starting to load Subject7 Data.
torch.Size([1, 32, 140424]) torch.Size([1, 6, 140424])
Finished! 1 data are loaded and preprocessed
Starting to load Subject8 Data.
torch.Size([1, 32, 124095]) torch.Size([1, 6

In [34]:
final_result_LSTM = makeTable(model = 'LSTM')
final_result_LSTM.to_csv(f'LSTM_final_result_padding.csv',index=False)

AUC:0.8208903186875063, Precision:0.47709593777009507, Recall:0.2164705882352941, F1:0.2978149446992177
AUC:0.9591796285843021, Precision:0.49693080357142855, Recall:0.6984313725490197, F1:0.5806977502445386
AUC:0.9586396367640639, Precision:0.4399951016409503, Recall:0.7045098039215686, F1:0.5416855118347655
AUC:0.9591266767682008, Precision:0.4633356465417534, Recall:0.7854901960784314, F1:0.5828604684999273
AUC:0.9362693821659178, Precision:0.5092918131592165, Recall:0.5964705882352941, F1:0.5494445949607153
AUC:0.8989677427019386, Precision:0.5679433368310598, Recall:0.4245098039215686, F1:0.4858617594254937
AUC:0.871567030070684, Precision:0.2507345739471107, Recall:0.1003921568627451, F1:0.14337720526463174
AUC:0.9178437789144447, Precision:0.3330906443392792, Recall:0.17941176470588235, F1:0.233210144004078
AUC:0.9057617737419107, Precision:0.3948509485094851, Recall:0.28568627450980394, F1:0.33151308304891924
AUC:0.9270093115087864, Precision:0.38433962264150945, Recall:0.39941

In [35]:
final_result_LSTM.mean()

Subject      6.500000
AUC          0.866543
Precision    0.348755
Recall       0.293282
F1           0.300844
dtype: float64

In [36]:
final_result_LSTM.groupby('Subject').aggregate({'AUC':'mean','Precision':'mean','Recall':'mean',
                                 'F1':'mean'}).round(3)

,AUC,Precision,Recall,F1
Subject,,,,
1,0.922,0.492,0.571,0.506
2,0.908,0.287,0.245,0.253
3,0.837,0.344,0.276,0.292
4,0.906,0.420,0.426,0.416
5,0.756,0.256,0.117,0.156
6,0.841,0.371,0.217,0.264
7,0.886,0.380,0.212,0.254
8,0.874,0.328,0.415,0.359
9,0.877,0.394,0.320,0.352
